In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import matplotlib.pyplot as plt
from google.colab import auth
from google.colab import drive
from google.colab import files
import pandas as pd
import numpy as np
import torch.nn.functional as F

#data_load
#(9961, 40, 16)
#(9961, 51)
train_lstm = np.load('/content/drive/MyDrive/BIG DATA/train_lstm.npy')
print(train_lstm.shape)
TF_lstm = np.load('/content/drive/MyDrive/BIG DATA/TF_lstm.npy')
print(TF_lstm.shape)
TFw = np.load('/content/drive/MyDrive/BIG DATA/TFw.npy')

gpu = 'cuda' if torch.cuda.is_available() else 'cpu'
print(gpu)
# for reproducibility
torch.manual_seed(777)
if gpu == 'cuda':
    torch.cuda.manual_seed_all(777)
train1 = torch.FloatTensor(train_lstm)
print(train1.shape)
train = train1[0:40000,:,:].to(gpu)
trainv = train1[40000:,:,:].to(gpu)

TF1 = torch.IntTensor(TF_lstm)
print(TF1.shape)
TF = TF1[0:40000]
TF = TF.type(torch.LongTensor).to(gpu)
# TF = torch.squeeze(TF,dim=0).to(gpu)
TFv = TF1[40000:]

TFv = TFv.type(torch.LongTensor).to(gpu)
# TFv = torch.squeeze(TFv).to(gpu)
# TFw1 = torch.FloatTensor(TFw)
# TFw = TFw1[0:40000,:]
# TFwv = TFw1[40000:,:]
#model sequence
class SingleRNN(nn.Module):

    def __init__(self, input_size, hidden_size, dropout=0, bidirectional=False):
        super(SingleRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_direction = int(bidirectional) + 1
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers = 3, dropout=dropout, batch_first=True, bidirectional=bidirectional)
        self.softmax = nn.Softmax(dim=-1)
        

    def forward(self, input):
        # input shape: batch, seq, dim
        output = input
        rnn_output, _ = self.rnn(output)
        rnn_output = rnn_output[:,-1,:]
        softmax = self.softmax(rnn_output)
        softmax = torch.squeeze(softmax, dim = 0)
        return softmax


      


model = SingleRNN(input_size=16,hidden_size = 1, dropout = 0, bidirectional= False).to(gpu)
output1 = model(train)
# modelv = SingleRNN(input_size=16,hidden_size = 51, dropout = 0, bidirectional= False)

optimizer = optim.Adam(model.parameters(), lr=0.01)
best_model_wts = model.state_dict() 
best_acc = 0
best_epoch = 0
criterion = nn.CrossEntropyLoss()
l_array = []
a_array = []
for epoch in range(1000):
  for i in range(2):
    if i == 0:
      output = model(train)
      # loss = (torch.abs((output - TF))).mean()
      print(TF)

      loss = criterion(output, TF)
      loss1 = loss.tolist()
      l_array.append(loss1)

      if (epoch + 1) % 100 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    else:
      pass
    
    if i == 1:
      pred = model(trainv)
      accuracy = criterion(pred,TFv)
      ac1 = accuracy.tolist()
      a_array.append(ac1)
      if best_acc > accuracy:
        best_model_wts = model.state_dict()
        best_acc = accuracy
        best_ephoch = epoch
      if (epoch+1) % 100 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'accuracy =', '{:.6f}'.format(accuracy))
    else:
      pass
print(best_acc)
print(best_ephoch)

plt.plot(l_array, 'r')
plt.plot(a_array,'b')
plt.show()
model_save_name = 'classifierL.pt'
path = F"/content/drive/My Drive/{model_save_name}" 
torch.save(model.state_dict(), path)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import matplotlib.pyplot as plt
from google.colab import auth
from google.colab import drive
from google.colab import files
import pandas as pd
import numpy as np
import torch.nn.functional as F
import numpy as np
import torch

loss = torch.nn.CrossEntropyLoss()

Y_pred1 = torch.Tensor([[2.0,1.0,0.1],
                       [1.0,1.4,0.1],
                       [1.1,0.2,3.1]])
Y_pred2 = torch.Tensor([[0.5,1.0,2.5],
                       [2.2,1.1,0.3],
                       [1.5,2.0,1.3],[1.5,2.0,1.3]])
print(Y_pred2.shape)

torch.Size([4, 3])
